In [9]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True )

learning_rate= 0.02
learning_rate_GradientDescent = 0.5
training_epochs= 100
batch_size = 256
display_step = 1
examples_to_show =10
input_size = 784
hidden1_size = 256
hidden2_size = 128
output_size = 10

x= tf.placeholder(tf.float32, shape= [None, input_size])
y= tf.placeholder(tf.float32, shape = [None, output_size])

def build_autoencoder(x):
    #인코딩
    W1= tf.Variable(tf.random_normal(shape= [input_size, hidden1_size]))
    b1= tf.Variable(tf.random_normal(shape= [hidden1_size]))
    H1_output = tf.nn.sigmoid(tf.matmul(x,W1)+ b1)
    
    W2= tf.Variable(tf.random_normal(shape= [hidden1_size, hidden2_size]))
    b2= tf.Variable(tf.random_normal(shape= [hidden2_size]))
    H2_output = tf.nn.sigmoid(tf.matmul(H1_output,W2)+ b2)
    
    #특징들을 압축하고 다시 원본데이터를 만듦 
    
    #디코딩
    W3= tf.Variable(tf.random_normal(shape= [hidden2_size, hidden1_size]))
    b3= tf.Variable(tf.random_normal(shape= [hidden1_size]))
    H3_output = tf.nn.sigmoid(tf.matmul(H2_output,W3)+ b3)

    W4= tf.Variable(tf.random_normal(shape= [hidden1_size, input_size]))
    b4= tf.Variable(tf.random_normal(shape= [input_size]))
    reconstructed_x = tf.nn.sigmoid(tf.matmul(H3_output,W4)+ b4)
    
    return reconstructed_x, H2_output

def fine_tuning_softmax(x):
    W_softmax = tf.Variable(tf.random_normal(shape= [hidden2_size, output_size]))
    b_softmax = tf.Variable(tf.random_normal(shape= [output_size]))
    model= tf.nn.softmax(tf.matmul(x,W_softmax)+ b_softmax)
    
    return model

y_pred, summarized_feature = build_autoencoder(x)
y_true = x

fine_pred = fine_tuning_softmax(summarized_feature)
#preprocessing

loss= tf.reduce_mean(tf.pow(y_true- y_pred, 2))
optimizer= tf.train.RMSPropOptimizer(learning_rate)
train_step = optimizer.minimize(loss)

#findtuning
fine_pred = fine_tuning_softmax(summarized_feature)
fine_loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(fine_pred), reduction_indices=[1])) 
finetuning_train_step = tf.train.GradientDescentOptimizer(learning_rate_GradientDescent).minimize(fine_loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    total_batch = int(mnist.train.num_examples/ batch_size)
    
    for epoch in range(training_epochs):
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _,current_loss = sess.run([train_step, loss], feed_dict = {x: batch_xs})
        
        if epoch%display_step ==0:
            print("반복: {}, 손실함수: {}".format(epoch+1, current_loss))
    
    for epoch in range(training_epochs+100):
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, current_loss= sess.run([finetuning_train_step, fine_loss], feed_dict = {x:batch_xs, y: batch_ys})
    
        if epoch%display_step ==0:
            print("반복: {}, 손실함수: {}".format(epoch+1, current_loss))
    
    
    correct_prediction = tf.equal(tf.argmax(fine_pred, 1), tf.argmax(y,1))
    accuracy =tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("accuracy: %f" %(accuracy.eval(feed_dict={x:mnist.test.images , y: mnist.test.labels})))
    

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
반복: 1, 손실함수: 0.22321924567222595
반복: 2, 손실함수: 0.16759510338306427
반복: 3, 손실함수: 0.1442367434501648
반복: 4, 손실함수: 0.1341223567724228
반복: 5, 손실함수: 0.12098102271556854
반복: 6, 손실함수: 0.1147533431649208
반복: 7, 손실함수: 0.10860934853553772
반복: 8, 손실함수: 0.10383854061365128
반복: 9, 손실함수: 0.09903304278850555
반복: 10, 손실함수: 0.09233719110488892
반복: 11, 손실함수: 0.09402557462453842
반복: 12, 손실함수: 0.08987582474946976
반복: 13, 손실함수: 0.08966292440891266
반복: 14, 손실함수: 0.08673716336488724
반복: 15, 손실함수: 0.08461609482765198
반복: 16, 손실함수: 0.08402477204799652
반복: 17, 손실함수: 0.08344720304012299
반복: 18, 손실함수: 0.08146122843027115
반복: 19, 손실함수: 0.07931762933731079
반복: 20, 손실함수: 0.08148158341646194
반복: 21, 손실함수: 0.08045832067728043
반복: 22, 손실함수: 0.0790647566318512
반복: 23, 손실함수: 0.07771112769842148
반복: 24, 손실함수: 0.07771184295415878
반복: 25,

반복: 136, 손실함수: 0.0188621636480093
반복: 137, 손실함수: 0.013431376777589321
반복: 138, 손실함수: 0.010004465468227863
반복: 139, 손실함수: 0.044416677206754684
반복: 140, 손실함수: 0.013999395072460175
반복: 141, 손실함수: 0.0091487355530262
반복: 142, 손실함수: 0.011671110056340694
반복: 143, 손실함수: 0.01377631351351738
반복: 144, 손실함수: 0.01962006464600563
반복: 145, 손실함수: 0.011507892049849033
반복: 146, 손실함수: 0.008837124332785606
반복: 147, 손실함수: 0.016159454360604286
반복: 148, 손실함수: 0.013342885300517082
반복: 149, 손실함수: 0.019755171611905098
반복: 150, 손실함수: 0.010456377640366554
반복: 151, 손실함수: 0.012785488739609718
반복: 152, 손실함수: 0.011430926620960236
반복: 153, 손실함수: 0.01149975135922432
반복: 154, 손실함수: 0.013453997671604156
반복: 155, 손실함수: 0.010278180241584778
반복: 156, 손실함수: 0.007542074657976627
반복: 157, 손실함수: 0.0085756191983819
반복: 158, 손실함수: 0.009154612198472023
반복: 159, 손실함수: 0.009883670136332512
반복: 160, 손실함수: 0.009346514008939266
반복: 161, 손실함수: 0.01188852358609438
반복: 162, 손실함수: 0.011174840852618217
반복: 163, 손실함수: 0.012071982026100159
반복